Quickstart

In [2]:
# 데이터 작업
# 데이터 작업을 위한 두 가지 기본 요소 : torch.utils.data.DataLoader , torch.utils.data.Dataset. 
# Dataset샘플과 해당 레이블을 저장 DataLoader

import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

# PyTorch는 TorchText , TorchVision 및 TorchAudio 와 같은 도메인별 라이브러리를 제공하며 모두 데이터 세트를 포함
# 이 튜토리얼에서는 TorchVision 데이터 세트를 사용

In [3]:
# torchvision.datasets모듈에는 CIFAR, COCO와 같은 많은 실제 비전 데이터에 대한 개체가 포함되어 있음
# 이 튜토리얼에서는 FashionMNIST 데이터세트를 사용 
# 모든 TorchVision에는 샘플과 라벨을 각각 수정하는 두 가지 인수가 포함되어 있음 Dataset transform target_transform

# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [4]:
# Dataset를 인수로 전달 DataLoader. 
# : 데이터 세트에 대한 반복 가능 항목을 래핑하고 자동 일괄 처리, 샘플링, 셔플링 및 다중 프로세스 데이터 로드를 지원
# 여기서는 배치 크기를 64로 정의
# 즉, iterable 데이터로더의 각 요소는 64개의 기능과 레이블로 구성된 배치를 반환

batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [5]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [6]:
# Define model 모델 생성

# PyTorch에서 신경망을 정의하기 위해 nn.Module을 상속하는 클래스를 만든다
# 함수에서 네트워크의 계층을 정의 __init__하고 
# 함수에서 데이터가 네트워크를 통과하는 방법을 지정 (forward) 
# 신경망의 작업을 가속화하기 위해 가능한 경우 GPU 또는 MPS로 이동

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [7]:
# 모델 매개변수 최적화
# 모델을 훈련하려면 손실 함수와 최적화 프로그램이 필요

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [8]:
# 단일 훈련 루프에서 모델은 훈련 데이터세트(일괄적으로 입력)에 대해 예측을 수행하고 예측 오류를 역전파하여 모델의 매개변수를 조정

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [9]:
# 또한 테스트 데이터 세트와 비교하여 모델 성능을 확인하여 학습 중인지 확인

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [10]:
# 훈련 과정은 여러 반복( epoch )에 걸쳐 수행
# 각 epoch마다 모델은 더 나은 예측을 위해 매개변수를 학습
# 에포크를 세대라고 번역하기도 함 (책에서도 세대라는 용어 나왔음)
# 각 epoch마다 모델의 정확도와 손실을 인쇄
# 매 에포크마다 정확도가 증가하고 손실이 감소하는 것을 예상

epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.302471  [   64/60000]
loss: 2.287063  [ 6464/60000]
loss: 2.271568  [12864/60000]
loss: 2.265818  [19264/60000]
loss: 2.237872  [25664/60000]
loss: 2.219079  [32064/60000]
loss: 2.229588  [38464/60000]
loss: 2.188260  [44864/60000]
loss: 2.185202  [51264/60000]
loss: 2.164438  [57664/60000]
Test Error: 
 Accuracy: 49.1%, Avg loss: 2.146684 

Epoch 2
-------------------------------
loss: 2.156263  [   64/60000]
loss: 2.142411  [ 6464/60000]
loss: 2.081804  [12864/60000]
loss: 2.099488  [19264/60000]
loss: 2.039707  [25664/60000]
loss: 1.988637  [32064/60000]
loss: 2.025023  [38464/60000]
loss: 1.930287  [44864/60000]
loss: 1.942073  [51264/60000]
loss: 1.880179  [57664/60000]
Test Error: 
 Accuracy: 56.1%, Avg loss: 1.863148 

Epoch 3
-------------------------------
loss: 1.899779  [   64/60000]
loss: 1.867910  [ 6464/60000]
loss: 1.740246  [12864/60000]
loss: 1.785234  [19264/60000]
loss: 1.672924  [25664/60000]
loss: 1.635085  [32064/600

In [11]:
# 모델 저장
# 모델을 저장하는 일반적인 방법은 내부 상태 사전(모델 매개변수 포함)을 직렬화하는 것
# internal state dictionary 내부 상태 사전으로 번역됨

torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [12]:
# 모델 로드
# 모델을 로드하는 프로세스에는 모델 구조를 다시 만들고 상태 사전을 해당 구조에 로드하는 작업이 포함

model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [13]:
# 이제 이 모델을 사용하여 예측

classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
